This notebook is for descriptive analysis purposes of the dataset

In [28]:
import kagglehub
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import torchvision.models as models
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from torchvision.models import resnet18, ResNet18_Weights
import numpy as np

path = kagglehub.dataset_download("jessicali9530/caltech256")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# There are 256 object categories
num_classes = 256
num_epochs = 10
learning_rate = 0.001

# Transformations for the images
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [29]:
# Loading the dataset
dataset = ImageFolder(root=path, transform=transform)

# Splitting into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# CNN to extract image features
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Processes the image. Outputs feature vector
        resnet = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        # Removes last classification layer
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
    
    def forward(self, x):
        with torch.no_grad():
            features = self.resnet(x)
            # Flatten then return
            return features.view(features.size(0), -1)

In [ ]:
model = CNN()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
n_total_steps = len(train_loader)

# sklearn models
random_forest = RandomForestClassifier()
decision_tree = DecisionTreeClassifier()
svc = SVC()

train_outputs = []
train_labels = []
# Looping through CNN and getting the outputs then
# Removing gradient tracking, convert to numpy, then
# Append to regular arrays to later fit into ML models
for images, labels in train_loader:
    output = model(images)
    train_outputs.append(output.detach().numpy())
    train_labels.append(labels.detach().numpy())
    
X = np.vstack(train_outputs)
y = np.hstack(train_labels)

random_forest.fit(X, y)
decision_tree.fit(X, y)
svc.fit(X, y) #The number of classes has to be greater than one; got 1 class

ValueError: The number of classes has to be greater than one; got 1 class

In [ ]:
test_outputs = []
test_labels = []

for images, labels in train_loader:
    output = model(images)
    test_outputs.append(output.numpy())
    test_labels.append(labels.numpy())

X = np.vstack(test_outputs)
y = np.hstack(test_labels)

r_pred = random_forest.predict(X)
d_pred = decision_tree.predict(X)
s_pred = svc.predict(X) #'SVC' object has no attribute 'support_vectors_'

print(accuracy_score(y, r_pred))
print(accuracy_score(y, d_pred))
print(accuracy_score(y, s_pred))

AttributeError: 'SVC' object has no attribute 'support_vectors_'